In [216]:
from sklearn import datasets
from sklearn import ensemble
data_breast_cancer_X, data_breast_cancer_y = datasets.load_breast_cancer(return_X_y= True, as_frame=True)
data_breast_cancer_X = data_breast_cancer_X[['mean texture','mean symmetry']]
data_breast_cancer_X
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_breast_cancer_X, data_breast_cancer_y, test_size=0.2)

In [217]:
# tworzenie estymatorów 
# 1.drzewa 
# 2.regresja logistyczna
# 3.knn

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

tree_clf = DecisionTreeClassifier()
log_clf = LogisticRegression()
knn_clf = KNeighborsClassifier()

voting_clf_hard = VotingClassifier(
    estimators=[('tree', tree_clf), ('log', log_clf), ('knn', knn_clf)],
    voting='hard'
)
voting_clf_soft = VotingClassifier(
    estimators=[('tree', tree_clf), ('log', log_clf), ('knn', knn_clf)],
    voting='soft'
)

In [218]:
voting_clf_hard.fit(X_train, y_train)
voting_clf_soft.fit(X_train, y_train)

VotingClassifier(estimators=[('tree', DecisionTreeClassifier()),
                             ('log', LogisticRegression()),
                             ('knn', KNeighborsClassifier())],
                 voting='soft')

In [219]:
#liczenie dokładności dla głosowanie hard i sofr
from sklearn.metrics import accuracy_score

y_test_predict_hard = voting_clf_hard.predict(X_test)
y_test_predict_soft = voting_clf_soft.predict(X_test)

y_train_predict_hard = voting_clf_hard.predict(X_train)
y_train_predict_soft = voting_clf_soft.predict(X_train)

y_test_predict_hard_score = accuracy_score(y_test, y_test_predict_hard)
y_test_predict_soft_score = accuracy_score(y_test, y_test_predict_soft)

y_train_predict_hard_score = accuracy_score(y_train, y_train_predict_hard)
y_train_predict_soft_score = accuracy_score(y_train, y_train_predict_soft)

voting_clf_hard_scores = [y_train_predict_hard_score, y_test_predict_hard_score]
voting_clf_soft_scores = [y_train_predict_soft_score, y_test_predict_soft_score]

print(voting_clf_hard_scores, voting_clf_soft_scores)

[0.8571428571428571, 0.7192982456140351] [0.9714285714285714, 0.6578947368421053]


In [221]:
# dokładności dla kazdego z samych estymatorów

for estimator in {tree_clf, log_clf, knn_clf}:
    accuracy_score_train = accuracy_score(y_train, estimator.predict(X_train))
    accuracy_score_test = accuracy_score(y_test, estimator.predict(X_test))
    print(accuracy_score_train, accuracy_score_test)

NotFittedError: This KNeighborsClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
# random forest 
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=30, max_leaf_nodes=12, random_state=42)
rnd_clf.fit(X_train, y_train)

RandomForestClassifier(max_leaf_nodes=12, n_estimators=30, random_state=42)